In [1]:
import warnings

warnings.filterwarnings("ignore")

import copy
import json
import os
import re
import sys
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from scipy.ndimage import convolve
from tqdm import tqdm

In [8]:
# MAP:
# 01 02 03 04 05 06 07 08 09 10 11
#       12    13    14    15
#       16    17    18    19


class State:
    def __init__(self, ani_pos, cum_energy=0):
        self.all_ani = ["a1", "a2", "b1", "b2", "c1", "c2", "d1", "d2"]
        self.ani_pos = ani_pos.copy()
        self.cum_energy = cum_energy
        self.diagram = self.convert_to_diagram()
        self.in_final_pos = self.get_already_in_final_pos()
        self.unlocked_ani = self.get_unlocked_ani()
        self.all_hallway_pos = np.array([1, 2, 4, 6, 8, 10, 11])
        self.hallway_taken_pos = np.array(
            [pos for pos in self.all_hallway_pos if self.diagram[pos] != ""]
        )

    def print_key_info(self):
        print(f"cum_energy: {self.cum_energy}")
        print(f"in_final_pos: {self.in_final_pos}")
        print(f"unlocked_ani: {self.unlocked_ani}")
        print(f"hallway_taken_pos: {self.hallway_taken_pos}")

    def __repr__(self):
        info = ""
        for pos in range(1, 12):
            info += self.diagram[pos] if self.diagram[pos] != "" else f"{pos:02d}"
            info += " "
        info += "\n      "
        for pos in range(12, 16):
            info += self.diagram[pos] if self.diagram[pos] != "" else f"{pos:02d}"
            info += "    "
        info += "\n      "
        for pos in range(16, 20):
            info += self.diagram[pos] if self.diagram[pos] != "" else f"{pos:02d}"
            info += "    "
        return info

    def check_if_finish(self):
        return True if len(self.in_final_pos) == 8 else False

    def get_next_state(self):
        new_states = []
        for ani in self.unlocked_ani:
            pos = self.ani_pos[ani]
            final_pos = self.check_if_accessible_to_final_pos(ani)
            if final_pos > 0:
                new_ani_pos = self.ani_pos.copy()
                new_ani_pos[ani] = final_pos
                new_energy = self.calculate_eng(pos, final_pos, ani)
                new_states.append(State(new_ani_pos, self.cum_energy + new_energy))
        # go to room first.
        # if no possible to go room, then go to hallway.
        if len(new_states) == 0:
            for ani in self.unlocked_ani:
                pos = self.ani_pos[ani]
                if pos > 11:
                    possible_hallway_pos = self.get_possible_hallway_pos(pos)
                    for new_pos in possible_hallway_pos:
                        new_ani_pos = self.ani_pos.copy()
                        new_ani_pos[ani] = new_pos
                        new_energy = self.calculate_eng(pos, new_pos, ani)
                        new_states.append(
                            State(new_ani_pos, self.cum_energy + new_energy)
                        )

        return new_states

    def distance_to_hallway(self, pos):
        if 12 <= pos <= 15:
            return 1
        if 16 <= pos <= 19:
            return 2
        if 20 <= pos <= 23:
            return 3
        if 24 <= pos <= 27:
            return 4

    def calculate_eng(self, start_pos, end_pos, ani):
        eng_by_step = self.get_eng_by_step(ani)

        step_num = 0
        if start_pos > 11:
            start_hallway_pos = self.map_room_to_hallway(start_pos)
            step_num += self.distance_to_hallway(start_pos)
        else:
            start_hallway_pos = start_pos
        if end_pos > 11:
            end_hallway_pos = self.map_room_to_hallway(end_pos)
            step_num += self.distance_to_hallway(end_pos)
        else:
            end_hallway_pos = end_pos

        step_num += abs(end_hallway_pos - start_hallway_pos)
        return step_num * eng_by_step

    def get_eng_by_step(self, ani):
        ani_type = ani[0]
        if ani_type == "a":
            return 1
        if ani_type == "b":
            return 10
        if ani_type == "c":
            return 100
        if ani_type == "d":
            return 1000

    def check_if_accessible_to_final_pos(self, ani):
        possible_final_pos = self.get_final_pos(ani)
        if possible_final_pos == 0:
            return 0
        else:
            if self.check_if_accessible_to_room(self.ani_pos[ani], possible_final_pos):
                return possible_final_pos
            return 0

    def get_col_ani(self, pos):
        if pos in [12, 16, 20, 24]:
            return "a"
        elif pos in [13, 17, 21, 25]:
            return "b"
        elif pos in [14, 18, 22, 26]:
            return "c"
        else:
            return "d"

    def get_bottom_pos(self, ani):
        ani_type = ani[0]
        if ani_type == "a":
            bottom_pos = 16
        elif ani_type == "b":
            bottom_pos = 17
        elif ani_type == "c":
            bottom_pos = 18
        elif ani_type == "d":
            bottom_pos = 19
        return bottom_pos

    def get_final_pos(self, ani):
        bottom_pos = self.get_bottom_pos(ani)

        ani_type = ani[0]

        if self.diagram[bottom_pos] == "":
            return bottom_pos
        elif (self.diagram[bottom_pos][0] == ani_type) and (
            self.diagram[bottom_pos - 4] == ""
        ):
            return bottom_pos - 4
        return 0

    def map_room_to_hallway(self, pos):
        # pos should be >=12
        if pos in [12, 16, 20, 24]:
            hallway_pos = 3
        elif pos in [13, 17, 21, 25]:
            hallway_pos = 5
        elif pos in [14, 18, 22, 26]:
            hallway_pos = 7
        else:
            hallway_pos = 9
        return hallway_pos

    def check_if_accessible_to_room(self, pos, room):
        hallway_pos_end = self.map_room_to_hallway(room)
        hallway_pos_start = self.map_room_to_hallway(pos) if pos > 11 else pos

        left_hallway_pos = min(hallway_pos_start, hallway_pos_end)
        right_hallway_pos = max(hallway_pos_start, hallway_pos_end)

        need_path_pos = self.all_hallway_pos[
            (self.all_hallway_pos > left_hallway_pos)
            & (self.all_hallway_pos < right_hallway_pos)
        ]
        if len(np.intersect1d(need_path_pos, self.hallway_taken_pos)):
            return False
        return True

    def get_possible_hallway_pos(self, pos):
        # pos should be in [12,..., 19]
        hallway_pos = self.map_room_to_hallway(pos)

        # must be smaller than this value
        right_bound = int(
            np.concatenate(
                (self.hallway_taken_pos[self.hallway_taken_pos > hallway_pos], [12])
            ).min()
        )
        # must be bigger than this value
        left_bound = int(
            np.concatenate(
                (self.hallway_taken_pos[self.hallway_taken_pos < hallway_pos], [0])
            ).max()
        )

        return self.all_hallway_pos[
            (self.all_hallway_pos > left_bound) & (self.all_hallway_pos < right_bound)
        ]

    def get_unlocked_ani(self):
        locked_ani = self.in_final_pos[:]

        for pos in range(12, 16):
            if self.diagram[pos] != "":
                locked_ani.extend(
                    [
                        self.diagram[pos + 4],
                    ]
                )

        unlocked_ani = [ani for ani in self.all_ani if ani not in set(locked_ani)]
        return unlocked_ani

    def convert_to_diagram(self):
        diagram = {i + 1: "" for i in range(19)}
        for ani, pos in self.ani_pos.items():
            diagram[pos] = ani
        return diagram

    def get_already_in_final_pos(self):
        in_final_pos = []
        for pos in range(16, 20):
            ani_type = self.get_col_ani(pos)
            if (self.diagram[pos] != "") and (self.diagram[pos][0] == ani_type):
                in_final_pos.append(self.diagram[pos])
                if (self.diagram[pos - 4] != "") and (
                    self.diagram[pos - 4][0] == ani_type
                ):
                    in_final_pos.append(self.diagram[pos - 4])
        return in_final_pos

In [9]:
%%time
current_smallest_eng = 9999999999

def play(state):
    global current_smallest_eng
    for new_state in state.get_next_state():
        new_eng = new_state.cum_energy
        if new_state.check_if_finish():
            if new_eng < current_smallest_eng:
                current_smallest_eng = new_eng
                print(f'Current smallest eng: {current_smallest_eng}')
        elif new_eng < current_smallest_eng:
            play(new_state)


play(
    State(
        {"a1": 12, "a2": 18, "b1": 16, "b2": 19, "c1": 13, "c2": 14, "d1": 17, "d2": 15}
    )
)
print("--------------")
print(f"Answer to Q1: {current_smallest_eng}")

Current smallest eng: 19286

Current smallest eng: 15286

Current smallest eng: 15266

Current smallest eng: 13286

Current smallest eng: 13086

...

Current smallest eng: 13072

Current smallest eng: 13066

--------------

Answer to Q1: 13066

Wall time: 8min 24s


In [4]:
# MAP:
# 01 02 03 04 05 06 07 08 09 10 11
#       12    13    14    15
#       16    17    18    19
#       20    21    22    23
#       24    25    26    27


class State2:
    def __init__(self, ani_pos, cum_energy=0):
        self.all_ani = [
            "a1",
            "a2",
            "a3",
            "a4",
            "b1",
            "b2",
            "b3",
            "b4",
            "c1",
            "c2",
            "c3",
            "c4",
            "d1",
            "d2",
            "d3",
            "d4",
        ]
        self.ani_pos = ani_pos.copy()
        self.cum_energy = cum_energy
        self.diagram = self.convert_to_diagram()
        self.in_final_pos = self.get_already_in_final_pos()
        self.unlocked_ani = self.get_unlocked_ani()
        self.all_hallway_pos = np.array([1, 2, 4, 6, 8, 10, 11])
        self.hallway_taken_pos = np.array(
            [pos for pos in self.all_hallway_pos if self.diagram[pos] != ""]
        )

    def print_key_info(self):
        print(f"cum_energy: {self.cum_energy}")
        print(f"in_final_pos: {self.in_final_pos}")
        print(f"unlocked_ani: {self.unlocked_ani}")
        print(f"hallway_taken_pos: {self.hallway_taken_pos}")

    def __repr__(self):
        info = ""
        for pos in range(1, 12):
            info += self.diagram[pos] if self.diagram[pos] != "" else f"{pos:02d}"
            info += " "
        info += "\n      "
        for pos in range(12, 16):
            info += self.diagram[pos] if self.diagram[pos] != "" else f"{pos:02d}"
            info += "    "
        info += "\n      "
        for pos in range(16, 20):
            info += self.diagram[pos] if self.diagram[pos] != "" else f"{pos:02d}"
            info += "    "
        info += "\n      "
        for pos in range(20, 24):
            info += self.diagram[pos] if self.diagram[pos] != "" else f"{pos:02d}"
            info += "    "
        info += "\n      "
        for pos in range(24, 28):
            info += self.diagram[pos] if self.diagram[pos] != "" else f"{pos:02d}"
            info += "    "
        return info

    def check_if_finish(self):
        return True if len(self.in_final_pos) == 16 else False

    def get_next_state(self):
        new_states = []
        for ani in self.unlocked_ani:
            pos = self.ani_pos[ani]
            final_pos = self.check_if_accessible_to_final_pos(ani)
            if final_pos > 0:
                new_ani_pos = self.ani_pos.copy()
                new_ani_pos[ani] = final_pos
                new_energy = self.calculate_eng(pos, final_pos, ani)
                new_states.append(State2(new_ani_pos, self.cum_energy + new_energy))
        # go to room first.
        # if no possible to go room, then go to hallway.
        if len(new_states) == 0:
            for ani in self.unlocked_ani:
                pos = self.ani_pos[ani]
                if pos > 11:
                    possible_hallway_pos = self.get_possible_hallway_pos(pos)
                    for new_pos in possible_hallway_pos:
                        new_ani_pos = self.ani_pos.copy()
                        new_ani_pos[ani] = new_pos
                        new_energy = self.calculate_eng(pos, new_pos, ani)
                        new_states.append(
                            State2(new_ani_pos, self.cum_energy + new_energy)
                        )

        return new_states

    def distance_to_hallway(self, pos):
        if 12 <= pos <= 15:
            return 1
        if 16 <= pos <= 19:
            return 2
        if 20 <= pos <= 23:
            return 3
        if 24 <= pos <= 27:
            return 4

    def calculate_eng(self, start_pos, end_pos, ani):
        eng_by_step = self.get_eng_by_step(ani)

        step_num = 0
        if start_pos > 11:
            start_hallway_pos = self.map_room_to_hallway(start_pos)
            step_num += self.distance_to_hallway(start_pos)
        else:
            start_hallway_pos = start_pos
        if end_pos > 11:
            end_hallway_pos = self.map_room_to_hallway(end_pos)
            step_num += self.distance_to_hallway(end_pos)
        else:
            end_hallway_pos = end_pos

        step_num += abs(end_hallway_pos - start_hallway_pos)
        return step_num * eng_by_step

    def get_eng_by_step(self, ani):
        ani_type = ani[0]
        if ani_type == "a":
            return 1
        if ani_type == "b":
            return 10
        if ani_type == "c":
            return 100
        if ani_type == "d":
            return 1000

    def check_if_accessible_to_final_pos(self, ani):
        possible_final_pos = self.get_final_pos(ani)
        if possible_final_pos == 0:
            return 0
        else:
            if self.check_if_accessible_to_room(self.ani_pos[ani], possible_final_pos):
                return possible_final_pos
            return 0

    def get_col_ani(self, pos):
        if pos in [12, 16, 20, 24]:
            return "a"
        elif pos in [13, 17, 21, 25]:
            return "b"
        elif pos in [14, 18, 22, 26]:
            return "c"
        else:
            return "d"

    def get_bottom_pos(self, ani):
        ani_type = ani[0]
        if ani_type == "a":
            bottom_pos = 24
        elif ani_type == "b":
            bottom_pos = 25
        elif ani_type == "c":
            bottom_pos = 26
        elif ani_type == "d":
            bottom_pos = 27
        return bottom_pos

    def get_final_pos(self, ani):
        bottom_pos = self.get_bottom_pos(ani)

        ani_type = ani[0]

        if self.diagram[bottom_pos] == "":
            return bottom_pos
        elif (self.diagram[bottom_pos][0] == ani_type) and (
            self.diagram[bottom_pos - 4] == ""
        ):
            return bottom_pos - 4
        elif (
            (self.diagram[bottom_pos][0] == ani_type)
            and (self.diagram[bottom_pos - 4][0] == ani_type)
            and (self.diagram[bottom_pos - 8] == "")
        ):
            return bottom_pos - 8
        elif (
            (self.diagram[bottom_pos][0] == ani_type)
            and (self.diagram[bottom_pos - 4][0] == ani_type)
            and (self.diagram[bottom_pos - 8][0] == ani_type)
            and (self.diagram[bottom_pos - 12] == "")
        ):
            return bottom_pos - 12
        return 0

    def map_room_to_hallway(self, pos):
        # pos should be >=12
        if pos in [12, 16, 20, 24]:
            hallway_pos = 3
        elif pos in [13, 17, 21, 25]:
            hallway_pos = 5
        elif pos in [14, 18, 22, 26]:
            hallway_pos = 7
        else:
            hallway_pos = 9
        return hallway_pos

    def check_if_accessible_to_room(self, pos, room):
        hallway_pos_end = self.map_room_to_hallway(room)
        hallway_pos_start = self.map_room_to_hallway(pos) if pos > 11 else pos

        left_hallway_pos = min(hallway_pos_start, hallway_pos_end)
        right_hallway_pos = max(hallway_pos_start, hallway_pos_end)

        need_path_pos = self.all_hallway_pos[
            (self.all_hallway_pos > left_hallway_pos)
            & (self.all_hallway_pos < right_hallway_pos)
        ]
        if len(np.intersect1d(need_path_pos, self.hallway_taken_pos)):
            return False
        return True

    def get_possible_hallway_pos(self, pos):
        # pos should be in [12,..., 19]
        hallway_pos = self.map_room_to_hallway(pos)

        # must be smaller than this value
        right_bound = int(
            np.concatenate(
                (self.hallway_taken_pos[self.hallway_taken_pos > hallway_pos], [12])
            ).min()
        )
        # must be bigger than this value
        left_bound = int(
            np.concatenate(
                (self.hallway_taken_pos[self.hallway_taken_pos < hallway_pos], [0])
            ).max()
        )

        return self.all_hallway_pos[
            (self.all_hallway_pos > left_bound) & (self.all_hallway_pos < right_bound)
        ]

    def get_unlocked_ani(self):
        locked_ani = self.in_final_pos[:]

        for pos in range(12, 16):
            if self.diagram[pos] != "":
                locked_ani.extend(
                    [
                        self.diagram[pos + 4],
                        self.diagram[pos + 8],
                        self.diagram[pos + 12],
                    ]
                )
            elif self.diagram[pos + 4] != "":
                locked_ani.extend([self.diagram[pos + 8], self.diagram[pos + 12]])
            elif self.diagram[pos + 8] != "":
                locked_ani.extend([self.diagram[pos + 12]])

        unlocked_ani = [ani for ani in self.all_ani if ani not in set(locked_ani)]
        return unlocked_ani

    def convert_to_diagram(self):
        diagram = {i + 1: "" for i in range(27)}
        for ani, pos in self.ani_pos.items():
            diagram[pos] = ani
        return diagram

    def get_already_in_final_pos(self):
        in_final_pos = []
        for pos in range(24, 28):
            ani_type = self.get_col_ani(pos)
            if (self.diagram[pos] != "") and (self.diagram[pos][0] == ani_type):
                in_final_pos.append(self.diagram[pos])
                if (self.diagram[pos - 4] != "") and (
                    self.diagram[pos - 4][0] == ani_type
                ):
                    in_final_pos.append(self.diagram[pos - 4])
                    if (self.diagram[pos - 8] != "") and (
                        self.diagram[pos - 8][0] == ani_type
                    ):
                        in_final_pos.append(self.diagram[pos - 8])
                        if (self.diagram[pos - 12] != "") and (
                            self.diagram[pos - 12][0] == ani_type
                        ):
                            in_final_pos.append(self.diagram[pos - 12])
        return in_final_pos

In [5]:
%%time
current_smallest_eng = 9999999999


def play(state):
    global current_smallest_eng
    for new_state in state.get_next_state():
        new_eng = new_state.cum_energy
        if new_state.check_if_finish():
            if new_eng < current_smallest_eng:
                current_smallest_eng = new_eng
                print(f'Current smallest eng: {current_smallest_eng}')
        elif new_eng < current_smallest_eng:
            play(new_state)


play(
    State2(
        {
            "a1": 12,
            "a2": 19,
            "a3": 22,
            "a4": 26,
            "b1": 18,
            "b2": 21,
            "b3": 24,
            "b4": 27,
            "c1": 13,
            "c2": 14,
            "c3": 17,
            "c4": 23,
            "d1": 15,
            "d2": 16,
            "d3": 20,
            "d4": 25,
        }
    )
)
print("--------------")
print(f"Answer to Q2: {current_smallest_eng}")

Current smallest eng: 60680

Current smallest eng: 52678

Current smallest eng: 48128

Current smallest eng: 47328

--------------

Answer to Q2: 47328

Wall time: 5min 7s
